In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, ReLU, Flatten, LSTM, Input, MaxPooling1D, Lambda, MaxPooling2D
from tensorflow.keras.layers import Input, Embedding, BatchNormalization, Concatenate, Reshape
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Conv2D, SpatialDropout1D
from tensorflow.keras.models import Sequential, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras import regularizers
from keras.utils import to_categorical
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

Lấy data từ 2 file X_train Y_train X_test và Y_test từ các file X_train_cleaned Y_train_cleaned X_test_cleaned Y_test_cleaned

In [2]:
X_tr=pd.read_csv('Twitter_2/X_train_cleaned.csv',header=None,encoding='latin1')
Y_tr=pd.read_csv('Twitter_2/Y_train_cleaned.csv',header=None)
X_t=pd.read_csv('Twitter_2/X_test_cleaned.csv',header=None,encoding='latin1')
Y_t=pd.read_csv('Twitter_2/Y_test_cleaned.csv',header=None)
X_train = X_tr[0].astype(str).tolist()
X_test = X_t[0].astype(str).tolist()


Chuẩn bị data để nhét vào mô hình chuẩn bị việc train mô hình

In [3]:
encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_tr)
Y_test=encoder.transform(Y_t)
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences=tokenizer.texts_to_sequences(X_train)
tr_x=pad_sequences(sequences,maxlen=50)
tr_y=to_categorical(Y_train)
sequences=tokenizer.texts_to_sequences(X_test)
t_x=pad_sequences(sequences,maxlen=50)
t_y=to_categorical(Y_test)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Tạo mô hình CNN để chạy data

In [10]:
embedding_dim=300
max_len=50
max_word=20000
batchsize=64
epochs=25
def model1():
    branch1 = Sequential()
    branch1.add(Embedding(max_word,embedding_dim,input_length=max_len))
    branch1.add(Conv1D(64,3,padding='same',activation='relu'))
    branch1.add(BatchNormalization())
    branch1.add(ReLU())
    branch1.add(Dropout(0.5))
    branch1.add(GlobalMaxPooling1D())
    input1 = Input(shape=(max_len,))
    x1=branch1(input1)
    branch2 = Sequential()
    branch2.add(Embedding(max_word, embedding_dim, input_length=max_len))
    branch2.add(Conv1D(64, 3, padding='same', activation='relu'))
    branch2.add(BatchNormalization())
    branch2.add(ReLU())
    branch2.add(Dropout(0.5))
    branch2.add(GlobalMaxPooling1D())
    input2 = Input(shape=(max_len,))
    x2=branch2(input2)
    concatenated = Concatenate()([x1, x2])
    hid_layer=Dense(128,activation='relu')(concatenated)
    drop_layer=Dropout(0.3)(hid_layer)
    outp=Dense(3, activation='softmax')(drop_layer)
    model=Model(inputs=[branch1.input,branch2.input],outputs=outp)
    model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy',Precision(),Recall()])
    return model

Chạy mô hình với data train

In [11]:
model=model1()
model.summary()


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_10      │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_4        │ (None, 64)        │  6,057,920 │ input_layer_8[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_5        │ (None, 64)        │  6,057,920 │ input_layer_10[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 128)       │          0 │ sequential_4[0][… │
│ (Concatenate)       │                   │            │ sequential_5[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │     16,512 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 128)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 3)         │        387 │ dropout_8[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,132,739 (46.28 MB)

 Trainable params: 12,132,483 (46.28 MB)

 Non-trainable params: 256 (1.00 KB)

In [12]:
history=model.fit([tr_x,tr_x], tr_y, epochs=epochs, batch_size=batchsize, validation_split=0.2)

Epoch 1/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 24s 64ms/step - accuracy: 0.4104 - loss: 1.3261 - precision_2: 0.4328 - recall_2: 0.2879 - val_accuracy: 0.5463 - val_loss: 1.0469 - val_precision_2: 0.9815 - val_recall_2: 0.0096
Epoch 2/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - accuracy: 0.5913 - loss: 0.8661 - precision_2: 0.6630 - recall_2: 0.4354 - val_accuracy: 0.6593 - val_loss: 0.8423 - val_precision_2: 0.8477 - val_recall_2: 0.2998
Epoch 3/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - accuracy: 0.6873 - loss: 0.7131 - precision_2: 0.7372 - recall_2: 0.6044 - val_accuracy: 0.6811 - val_loss: 0.7625 - val_precision_2: 0.8115 - val_recall_2: 0.4322
Epoch 4/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - accuracy: 0.7446 - loss: 0.6118 - precision_2: 0.7747 - recall_2: 0.6975 - val_accuracy: 0.7098 - val_loss: 0.7151 - val_precision_2: 0.7957 - val_recall_2: 0.5527
Epoch 5/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - accuracy: 0.7860 - loss: 0.5360 - precision_2: 0.8065 - recall_2

Thử dữ liệu với data test

In [14]:
pred_label=model.predict([t_x,t_x])
predicted=np.argmax(pred_label,axis=1)
true=np.argmax(t_y,axis=1)
print(classification_report(true, predicted))


111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
              precision    recall  f1-score   support

           0       0.64      0.69      0.66      1001
           1       0.62      0.63      0.63      1430
           2       0.76      0.70      0.73      1103

    accuracy                           0.67      3534
   macro avg       0.67      0.67      0.67      3534
weighted avg       0.67      0.67      0.67      3534

